# 3.2 RecyclerView - Selection

<img src="https://media1.giphy.com/media/7VOdnpyhosbBDIQPR0/giphy.gif?cid=790b76117ce18b657fd9003f6e29b87c453b5bfad7667dad&rid=giphy.gif&ct=g" width="200" />

## **Implementacja `RecyclerView`**

Naszą aplikację rozpoczniemy od utworzenia layoutu `main_activity.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    tools:context=".MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content" />

</LinearLayout>

Następnie dodajmy plik `rv_item.xml`, który będzie zawierał zdefiniowany layout pojedynczego elementu `RecyclerView`

<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:orientation="horizontal"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/numberText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="default text"
        android:textSize="24sp"
        android:layout_gravity="center"
        android:textAlignment="center"
        android:layout_margin="16dp"
        android:background="@drawable/item_background"
        android:textStyle="bold"/>

</LinearLayout>

Będziemy posiadali tylko jedno pole `TextView`, w który wyświetlimy liczbę. Przejdźmy do `MainActivitry` i dodajmy funkcję zwracającą listę elementów, które będziemy wyświetlać.

In [ ]:
    private fun createList(): List<Int>{
        return (0..50).map { it }
    }

Następnie zaimplementujmy klasę `NumberListAdapter`

In [ ]:
class NumberListAdapter(
    private val numberList: List<Int>,
) : RecyclerView.Adapter<NumberListAdapter.NumberListViewHolder>() {

    override fun onCreateViewHolder(
        parent: ViewGroup,
        viewType: Int
    ): NumberListViewHolder {
        return NumberListViewHolder(
            LayoutInflater.from(parent.context).inflate(
                R.layout.rv_item,
                parent,
                false
            )
        )
    }

    override fun onBindViewHolder(holder: NumberListViewHolder, position: Int) {
        holder.bind(numberList[position])
    }

    override fun getItemCount() = numberList.size

    class NumberListViewHolder(itemView: View) : RecyclerView.ViewHolder(itemView) {
        val number: TextView = itemView.findViewById(R.id.numberText)

        fun bind(value: Int) {
            number.text = value.toString()
        }
    }
}

Tym razem nasz `Adapter` nie przyjmuje `Context` w konstruktorze. Do instancji `Context`, w metodzie `onCreateViewHolder` odwołujemy się przez `parent.parent`. W klasie `ViewHolder` mamy funkcję `bind` łączącą dane z odpowiednimi polami layoutu, jest ona następnie wywołana w metodzie `onBindViewHolder`.

## **Implementacja `Selector`**

W pierwszym kroku przejdźmy do pliku `build.gradle` i w bloku `dependencies` dodajmy wpis

In [ ]:
    implementation 'androidx.recyclerview:recyclerview-selection:1.1.0'

Po zmodyfikowaniu skryptu `gradle` musimy wykonać synchronizację (**Sync Project with Gradle Files**). Chcemy uzyskać możliwość zaznaczania elementów i zmieniania ich właściwości. W pierwszym kroku musimy zdecydować się na typ klucza który będziemy wykorzystywać:
- `String` - wykorzystywany gdy mamy identyfikator `String`
- `Long` - wykorzystywamy gdy mamy identyfikator numeryczny
- `Parcelable` - gdy wykorzystujemy jako klucz np. `Uri`
W naszym przypadku wykorzystamy `Long` i właściwość `position` naszego `RecyclerView`.

Przechodzimy do klasy `NumberListAdapter`, w pierwszym kroku musimy ustawić `hasStableIds` - zmiana na `true` oznacza że każdy element może być reprezentowany przez unikalny klucz. Do konstruktora dodajemy

In [ ]:
    init {
        setHasStableIds(true)
    }

Następnie chcemy wykorzystać właściwość `position` jako identyfikator, w tym celu nadpisujemy metodę `getItemId`

In [ ]:
    override fun getItemId(position: Int): Long = position.toLong()

Kolejnym krokiem jest implementacja klasy `KeyProvider`, tutaj skorzystamy z domyślnej implementacji `StableIdKeyProvider`. Następnie musimy dostarczyć implementację `ItemDetailsLookup` - klasa ta zawiera informacje o zaznaczonych przez użytkownika elementach. Dodajmy nowy plik do projektu - `NumberItemDetailsLookup.kt`.

Klasa `NumberItemDetailsLookup` musi rozszerzać klasę `ItemDetailsLookup`

In [ ]:
class NumberItemDetailsLookup(private val recyclerView: RecyclerView) :
    ItemDetailsLookup<Long>()

W klasie będziemy potrzebować pola `RecyclerView` reprezentujący naszą listę. Samo zaznaczanie w tej bibliotece bazuje na `MotionEvent` który musimy zmapować na nasz `ViewHolder`. W tym celu musimy naspisać metodę `getItemDetails`

In [ ]:
    override fun getItemDetails(event: MotionEvent): ItemDetails<Long>?{}

W pierwszym kroku potrzebujemy dostać się do naszego `ViewHolder` - zapiszemy go w obiekcie `View`. Możemy się do niego dostać wywołując metodę `findChildViewHolder` z klasy `RecyclerView`

In [ ]:
        val view = recyclerView.findChildViewUnder(event.x, event.y)

Metoda ta przyjmuje dwa argumenty
- `x` pozycję horyzontalną wyrażoną w pikselach
- `y` pozycję wertykalną wyrażoną w pikselach

Przekazując instancję `MotionEvent` z wywołanymi metodami `x` i `y` możemy ustalić który element został zaznaczony. Następnie sprawdzamy czy udało się zwrócić `ViewHolder` w warunku `if`, jeżeli wszystko poszło pomyślnie metoda zwraca obiekt typu `ItemDetails`, w przeciwnym razie zwracamy `null`

In [ ]:
return (recyclerView.getChildViewHolder(view) as NumberListAdapter.NumberListViewHolder).getItemDetails()

Obiekt `ItemDetails` musi zawierać dwie informacje:
- `position` - pozycję elementu `RecyclerView`
- `selectionId` - unikalny identyfikator elementu `RecyclerView`

Aby otrzymać te elementy do klasy `NumberListAdapter.NumberViewHolder` dodajemy metodę `getItemDetails` zwracającą `ItemDetail`.

In [ ]:
        fun getItemDetails(): ItemDetailsLookup.ItemDetails<Long> =
            object : ItemDetailsLookup.ItemDetails<Long>() {
                override fun getPosition(): Int = adapterPosition
                override fun getSelectionKey(): Long = itemId
            }

Zwracany obiekt (anonimowy) musi implementować dwie metody. Aktualną pozycję wyciągamy przez wyołanie metody `adapterPosition` - metoda zwraca `Adapter` który jako ostatni był powiązany z tym `ViewHolder` lub `null` jeżeli takiego powiązanie nie było. W metodzie `getSelectionKey` zwracamy wywołanie metody `itemId` klasy `RecyclerView`.

Kolejnym krokiem będzie zmienienie koloru tła zaznaczonego elementu. Możemy to zdefiniować w pliku `xml`. Do folderu **res -> drawable** dodajemy nowy plik **Drawable Resource File** (`item_background.xml`), jako **root element** wybieramy `selector`. Chcemy zdefiniować dwa kolory i ustawić jeden z nich dla stanu aktywnego

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<selector xmlns:android="http://schemas.android.com/apk/res/android">
    <item android:drawable="@android:color/holo_blue_bright" android:state_activated="true" />
    <item android:drawable="@android:color/white" />
</selector>

Kolor stanu aktywnego ustawiamy przez zmianę domyślnej wartości `android:state_activated` na `true`. W ten sposób zmianę tła będziemy mieć połączoną z właściwością `isSelected` elementu. 

Żeby z niego skorzystać potrzebujemy jeszcze jednego elementu - `SelectionTracker`. W klasie `NumberListAdapter` dodajemy odpowiednie pole z `setterem`

In [ ]:
    lateinit var selectionTracker: SelectionTracker<Long>

`SelectionTracker` umożliwia śledzenie wszystkich elementów zaznaczonych przez użytkownika i pozwala sprawdzić czy dany element jest zaznaczony czy nie. Zmodyfikujmy metodę `bind` klasy `ViewHolder` aby umożliwić śledzenie - robimy to poprzez ustawienie pola `isActivated` na `itemView`

In [ ]:
        fun bind(value: Int, isActivated: Boolean = false) {
            number.text = value.toString()
            itemView.isActivated = isActivated
        }

Teraz musimy zmodyfikować wywołanie w metodzie `onBindViewHolder`

In [ ]:
    override fun onBindViewHolder(holder: NumberListViewHolder, position: Int) {
        holder.bind(numberList[position], selectionTracker.isSelected(position.toLong()))
    }

Na instancji `SelectionTracker` wywołujemy metodę `isSelected` sptawdzającą czy element o zadanym `id` jest zaznaczony przez użytkownika. Jako `id` przekazujemy `position` zrzutowany na `long`.

Ostatnim krokiem jest utworzenie `SelectionTracker` w klasie `MainActivity` i połączenie go z naszym `RecyclerView`. 

In [ ]:
        recyclerView.adapter = numberListAdapter
        val selectionTracker = SelectionTracker.Builder(
            "numberSelection",
            recyclerView,
            StableIdKeyProvider(recyclerView),
            NumberItemDetailsLookup(recyclerView),
            StorageStrategy.createLongStorage()
        ).withSelectionPredicate(
            SelectionPredicates.createSelectAnything()
        ).build()
        numberListAdapter.selectionTracker = selectionTracker

Tutaj istotna jest kolejność wykonania
- tworzymy `RecyclerView`
- powiązujemy `Adapter` z `RecyclerView`
- powiązujemy `SelectionTracker` z `RecyclerView`

Aby utworzyć `SelectionTracker` wykorzystujemy `Builder`

In [ ]:
val selectionTracker = SelectionTracker.Builder()

Przyjmuje on szereg argumentów
- `selectionId` - `String` jednoznacznie identyfikujący `SelectionTracker` dla danej aktywności
- `recyclerView` - instancja `RecyclerView` na którym wywołujemy `SelectionTracker`
- `keyProvider` - źródło kluczy po których rozróżniamy elementy - tutaj wykorzystaliśmy domyślną implementację klasy `StableIdKeyProvider`
- `detailsLookup` - źródło informacji o elementach `Recyclerview` (pozycja elementu, unikalny identyfikator elementu)
- `storage` - strategia przechowywania stanu - możemy przechować informację o tym, które elementy są aktualnie zaznaczone

Następnie wywołujemy funkcję `withSelectionPredicate`, która pozwala nam określić sposób w jaki stosujemy zaznaczenie (możemy określić ograniczenia). Tutaj wywołana jest metoda `createSelectionAnything` nie posiadająca żadnych ograniczeń - jednocześnie zezwala na zaznaczenie wielu elementów.


Na tym etapie możemy przetestować aplikację.

<img src="https://media1.giphy.com/media/7VOdnpyhosbBDIQPR0/giphy.gif?cid=790b76117ce18b657fd9003f6e29b87c453b5bfad7667dad&rid=giphy.gif&ct=g" width="200" />